In [1]:
import pandas as pd, numpy as np, os, math

In [3]:
outstanding_games = pd.read_csv("2020GamesLeft.csv")

In [4]:
allTheStats = pd.read_csv("../perfectlyCleaned_Data.csv")

In [7]:
allTheStats.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [8]:
outstanding_games.columns

Index(['Year', 'Week2', 'Day', 'Date', 'Time', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'TEAM_POINT_MARGIN', 'OPPONENTS_POINTS',
       'OPPONENTS_YARDS', 'OPPONENTS_TURNOVERS',
       'OPPONENTS_TURNOVER_DIFFERENTIAL', 'GAMEID', 'BINARY_WIN', 'Home_Team'],
      dtype='object')

## model1: OVERALL FPI (ESPN Team Eff Metric)

In [10]:
#model1: just OVERALL FPI (ESPN Team Eff Metric)
model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [11]:
from sklearn.model_selection import train_test_split
import sklearn.linear_model

In [15]:
model_1 = sklearn.linear_model.LinearRegression()

In [13]:
X = model_values.iloc[:,1].values
y = model_values.iloc[:,0].values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
X_train = X_train.reshape(-1, 1)
X_test = X_test.reshape(-1, 1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [17]:
model_1.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [19]:
#R_sq Score
model_1.score(X_train, y_train)

0.11160052247107721

In [21]:
model_1.score(X_test, y_test)

0.12100326484896597

In [22]:
model_1.intercept_

array([22.64165239])

In [23]:
model_1.coef_

array([[0.64681882]])

In [25]:
y_pred = model_1.predict(X_test)

In [26]:
sq_resid = (y_pred - y_test)**2

In [27]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

7.449358973567071

In [30]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_1, params, verbose=3)

In [31]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.131, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.121, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.131, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.045, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.105, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.131, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [33]:
grid.best_score_

0.10659290608447188

In [34]:
ys = grid.predict(X_test)

In [35]:
resids = (ys - y_test)**2

In [36]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

7.449358973567071

In [37]:
from sklearn.metrics import r2_score

In [39]:
r2_score(y_test, ys)

0.12100326484896595

## model2:
### OVERALL FPI + TEAM OFFENSIVE FPI + THE OPPOSING_TEAM'S DEFENSIVE FPI + TEAM SPECIAL TEAMS FPI + Strength of Schedule FPI + WP Ranks

In [40]:
allTheStats.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [43]:
allTheStats2 = allTheStats.copy(deep=True)

In [44]:
#model2: All the ESPN EFF METRICS
#model_values = allTheStats.loc[:, ["TEAM_POINTS", "Power Index FPI"]]

In [50]:
opponentDef = allTheStats.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [78]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [80]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP']]

In [74]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
dtypes: float64(7)
memory usage: 181.4 KB


In [79]:
model_values.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', 'Power Index DEF_Opp']]

,FINAL_TEAM_NAME,Power Index DEF,Power Index DEF_Opp
0,Detroit Lions,-2.4,-2.7
1,Chicago Bears,-2.7,-2.4
2,Chicago Bears,-2.7,-2.4
3,Detroit Lions,-2.4,-2.7
4,Washington Redskins,-2.2,-2.7
...,...,...,...
2897,Kansas City Chiefs,1.0,-0.2
2898,Oakland Raiders,-3.4,1.3
2899,New England Patriots,1.3,-3.4
2900,Buffalo Bills,-0.2,-3.4


In [87]:
model_2 = sklearn.linear_model.LinearRegression(normalize=True)

In [88]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [90]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [91]:
model_2.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [92]:
#R_sq Score
model_2.score(X_train, y_train)

0.2117845417196058

In [93]:
model_2.score(X_test, y_test)

0.19001364187854686

In [94]:
model_2.intercept_

array([24.06222075])

In [95]:
model_2.coef_

array([[ 0.04133081,  0.67166743, -0.95707838,  0.47373876,  0.00707634,
        -0.09042084]])

In [96]:
y_pred = model_2.predict(X_test)

In [97]:
sq_resid = (y_pred - y_test)**2

In [98]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

6.950467849849554

In [99]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_2, params, verbose=3)

In [100]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.245, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.191, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.228, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.209, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.153, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.245, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [101]:
grid.best_score_

0.20522667173526582

In [102]:
ys = grid.predict(X_test)

In [103]:
resids = (ys - y_test)**2

In [104]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

6.950467849849554

In [105]:
from sklearn.metrics import r2_score

In [106]:
r2_score(y_test, ys)

0.19001364187854686

## Model 3:
### Past FPI and Introducing Yards and Turnovers

In [107]:
allTheStats2.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [108]:
opponentDef = allTheStats2.loc[:, ["FINAL_TEAM_NAME", 'Power Index DEF', "GAMEID"]]

In [109]:
model_values=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [112]:
allTheStats3=pd.merge(allTheStats2, opponentDef, left_on=["GAMEID", "Opponent"], right_on=["GAMEID", "FINAL_TEAM_NAME"], suffixes=("", "_Opp"))

In [110]:
model_values=model_values.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [111]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
 7   TEAM_YARDS           2902 non-null   float64
 8   TEAM_TURNOVERS       2902 non-null   float64
dtypes: float64(9)
memory usage: 226.7 KB


In [113]:
model_3 = sklearn.linear_model.LinearRegression(normalize=True)

In [114]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [118]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [120]:
model_3.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [121]:
#R_sq Score
model_3.score(X_train, y_train)

0.4887859670361537

In [122]:
model_3.score(X_test, y_test)

0.5273383363155932

In [123]:
model_3.intercept_

array([4.07569457])

In [124]:
model_3.coef_

array([[ 0.08343974,  0.11082988, -0.34514904,  0.36145503,  0.01845262,
        -0.08814872,  0.0644413 , -1.81601954]])

In [125]:
y_pred = model_3.predict(X_test)

In [126]:
sq_resid = (y_pred - y_test)**2

In [127]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

5.284216415845227

In [128]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_3, params, verbose=3)

In [129]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.401, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.495, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.477, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.508, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.513, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.401, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [130]:
grid.best_score_

0.4785656440179504

In [131]:
ys = grid.predict(X_test)

In [132]:
resids = (ys - y_test)**2

In [133]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

5.284216415845227

In [134]:
from sklearn.metrics import r2_score

In [135]:
r2_score(y_test, ys)

0.5273383363155932

## Model 4:
### Could A Classifier Work For a Regression Problem (Random Forest)

In [136]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [137]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS"]]

In [138]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TEAM_POINTS          2902 non-null   float64
 1   Power Index FPI      2902 non-null   float64
 2   Power Index OFF      2902 non-null   float64
 3   Power Index DEF_Opp  2902 non-null   float64
 4   Power Index ST       2902 non-null   float64
 5   Ranks SOS            2902 non-null   float64
 6   Ranks AVGWP          2902 non-null   float64
 7   TEAM_YARDS           2902 non-null   float64
 8   TEAM_TURNOVERS       2902 non-null   float64
dtypes: float64(9)
memory usage: 226.7 KB


In [139]:
from sklearn.ensemble import RandomForestClassifier

In [140]:
rf = RandomForestClassifier(n_estimators=4000, criterion="entropy", max_depth=5)

In [141]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [142]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [143]:
rf.fit(X_train, y_train)

C:\Users\Kyle\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='entropy', max_depth=5, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=4000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [144]:
#R_sq Score
rf.score(X_train, y_train)

0.25505514705882354

In [145]:
rf.score(X_test, y_test)

0.08677685950413223

In [146]:
rf.classes_

array([ 0.,  3.,  5.,  6.,  7.,  8.,  9., 10., 11., 12., 13., 14., 15.,
       16., 17., 18., 19., 20., 21., 22., 23., 24., 25., 26., 27., 28.,
       29., 30., 31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41.,
       42., 43., 44., 45., 46., 47., 48., 49., 51., 52., 54., 55., 57.,
       59.])

In [152]:
y_pred = rf.predict(X_test)

In [153]:
sq_resid = (y_pred - y_test)**2

In [154]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

9.420603480333007

In [155]:
## STOP POINT FOR RF ##

In [162]:
from sklearn.model_selection import RandomizedSearchCV

In [163]:
# Grid Search
from sklearn.model_selection import RandomizedSearchCV
params = {'n_estimators':[4,40,400,4000,8000], 'criterion':["gini", "entropy"], "max_depth":[2, 5, 10]}
grid = RandomizedSearchCV(rf, params, verbose=3)

In [164]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] n_estimators=4000, max_depth=2, criterion=entropy ...............


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=4000, max_depth=2, criterion=entropy, score=0.078, total=  29.3s
[CV] n_estimators=4000, max_depth=2, criterion=entropy ...............


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   29.2s remaining:    0.0s
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=4000, max_depth=2, criterion=entropy, score=0.078, total=  31.3s
[CV] n_estimators=4000, max_depth=2, criterion=entropy ...............


...

[CV]  n_estimators=8000, max_depth=10, criterion=gini, score=0.074, total=  24.5s
[CV] n_estimators=8000, max_depth=10, criterion=gini .................


C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:515: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


[CV]  n_estimators=8000, max_depth=10, criterion=gini, score=0.092, total=  24.3s


[Parallel(n_jobs=1)]: Done  50 out of  50 | elapsed:  6.5min finished
C:\Users\Kyle\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:739: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='entropy',
                                                    max_depth=5,
                                                    max_features='auto',
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
            

In [165]:
grid.best_score_

0.09328271643994517

In [166]:
ys = grid.predict(X_test)

In [167]:
resids = (ys - y_test)**2

In [168]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

9.918588590639681

In [169]:
from sklearn.metrics import r2_score

In [170]:
r2_score(y_test, ys)

0.214525684460524

## Model 5:
### FPI, Yards, Turnovers + Expected Points Added per Play (EPA) and Success Rate for Offense overall and Pass EPA vs. Run EPA

In [171]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [173]:
model_values=allTheStats3.loc[:, ["TEAM_POINTS", "Power Index FPI", 'Power Index OFF','Power Index DEF_Opp', 'Power Index ST', 'Ranks SOS', 'Ranks AVGWP',
                                 "TEAM_YARDS", "TEAM_TURNOVERS",  'EPA/play_RDBMS_Winner', 'OVR_SUCCESS_RATE_RDBMS_Winner', 'Dropback EPA_RDBMS_Winner',
                                  'DROPBACK_SR_RDBMS_Winner', 'Rush EPA_RDBMS_Winner', 'RUN_SR_RDBMS_Winner']]

In [174]:
model_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2902 entries, 0 to 2901
Data columns (total 15 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   TEAM_POINTS                    2902 non-null   float64
 1   Power Index FPI                2902 non-null   float64
 2   Power Index OFF                2902 non-null   float64
 3   Power Index DEF_Opp            2902 non-null   float64
 4   Power Index ST                 2902 non-null   float64
 5   Ranks SOS                      2902 non-null   float64
 6   Ranks AVGWP                    2902 non-null   float64
 7   TEAM_YARDS                     2902 non-null   float64
 8   TEAM_TURNOVERS                 2902 non-null   float64
 9   EPA/play_RDBMS_Winner          2902 non-null   float64
 10  OVR_SUCCESS_RATE_RDBMS_Winner  2902 non-null   float64
 11  Dropback EPA_RDBMS_Winner      2902 non-null   float64
 12  DROPBACK_SR_RDBMS_Winner       2902 non-null   f

In [175]:
model_5= sklearn.linear_model.LinearRegression(normalize=True)

In [176]:
X = model_values.iloc[:,1:].values
y = model_values.iloc[:,0].values

In [177]:
X_train, X_test, y_train, y_test = train_test_split(X,y)
#X_train = X_train.reshape(-1)
#X_test = X_test.reshape(-1)
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

In [178]:
model_5.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [179]:
#R_sq Score
model_5.score(X_train, y_train)

0.6302700899587667

In [180]:
model_5.score(X_test, y_test)

0.6277712268609107

In [181]:
model_5.intercept_

array([19.35315188])

In [182]:
model_5.coef_

array([[ 1.30057265e-01, -8.14878619e-02, -2.34768717e-02,
         8.55711147e-01,  2.11939999e-02, -5.30486944e-02,
         2.24907272e-02,  5.33776096e-01,  2.75554251e+01,
        -3.30534744e+01,  4.57205456e+00,  1.49555454e+01,
         2.51527195e+00,  7.17865874e+00]])

In [204]:
model_values

,TEAM_POINTS,Power Index FPI,Power Index OFF,Power Index DEF_Opp,Power Index ST,Ranks SOS,Ranks AVGWP,TEAM_YARDS,TEAM_TURNOVERS,EPA/play_RDBMS_Winner,OVR_SUCCESS_RATE_RDBMS_Winner,Dropback EPA_RDBMS_Winner,DROPBACK_SR_RDBMS_Winner,Rush EPA_RDBMS_Winner,RUN_SR_RDBMS_Winner
0,24.0,-2.0,0.9,-2.7,-0.5,15.0,13.0,349.0,0.0,0.180,0.468,0.257,0.500,-0.041,0.375
1,20.0,-2.6,0.1,-2.4,0.0,12.0,27.0,345.0,3.0,-0.008,0.442,-0.009,0.483,-0.007,0.391
2,34.0,-2.6,0.1,-2.4,0.0,12.0,27.0,444.0,1.0,0.053,0.436,0.227,0.543,-0.198,0.281
3,37.0,-2.0,0.9,-2.7,-0.5,15.0,13.0,546.0,3.0,0.151,0.476,0.286,0.491,-0.095,0.448
4,24.0,-2.8,-0.4,-2.7,-0.2,25.0,10.0,374.0,1.0,0.195,0.551,0.428,0.676,-0.075,0.406
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2897,26.0,10.3,9.5,-0.2,-0.2,26.0,2.0,466.0,1.0,0.332,0.620,0.507,0.667,0.180,0.579
2898,20.0,-0.2,3.4,1.3,-0.1,1.0,20.0,375.0,3.0,-0.036,0.525,-0.001,0.564,-0.106,0.450
2899,36.0,-0.1,-1.8,-3.4,0.5,12.0,26.0,406.0,1.0,0.151,0.500,-0.057,0.469,0.346,0.529
2900,30.0,1.9,2.2,-3.4,-0.2,16.0,5.0,337.0,0.0,0.232,0.475,0.487,0.583,-0.169,0.304


In [183]:
y_pred = model_5.predict(X_test)

In [184]:
sq_resid = (y_pred - y_test)**2

In [185]:
# The Mean Squared Error
pd.Series(sq_resid.reshape(-1)).apply(math.sqrt).mean()

4.647672409034977

In [186]:
# Grid Search
from sklearn.model_selection import GridSearchCV
params = {'fit_intercept':[True,False], 'normalize':[True,False], 'copy_X':[True, False]}
grid = GridSearchCV(model_3, params, verbose=3)

In [189]:
# Train the model with GridSearch
grid.fit(X_train,y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.582, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.616, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.647, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.607, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=True .................
[CV]  copy_X=True, fit_intercept=True, normalize=True, score=0.645, total=   0.0s
[CV] copy_X=True, fit_intercept=True, normalize=False ................
[CV]  copy_X=True, fit_intercept=True, normalize=False, score=0.582, total=   0.0s
[CV] copy_X=True, fit

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  40 out of  40 | elapsed:    0.0s finished


GridSearchCV(cv=None, error_score=nan,
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=True),
             iid='deprecated', n_jobs=None,
             param_grid={'copy_X': [True, False],
                         'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=3)

In [190]:
grid.best_score_

0.6193625275279359

In [191]:
ys = grid.predict(X_test)

In [192]:
resids = (ys - y_test)**2

In [193]:
pd.Series(resids.reshape(-1)).apply(math.sqrt).mean()

4.647672409034978

In [194]:
from sklearn.metrics import r2_score

In [195]:
r2_score(y_test, ys)

0.6277712268609108

In [197]:
!cd ..

In [202]:
!ls

2020GamesLeft.csv
MachineLearningModels.ipynb


In [203]:
allTheStats3.columns

Index(['Unnamed: 0', 'Year', 'Week_x', 'Abbr_RDBMS_Winner', 'Week2', 'Day',
       'Date', 'Time', 'Home_Team', 'FINAL_TEAM_NAME', 'Opponent',
       'TEAM_POINTS', 'TEAM_YARDS', 'TEAM_TURNOVERS',
       'TEAM_TURNOVER_DIFFERENTIAL', 'OPPONENTS_POINTS', 'OPPONENTS_YARDS',
       'OPPONENTS_TURNOVERS', 'OPPONENTS_TURNOVER_DIFFERENTIAL',
       'TEAM_POINT_MARGIN', 'GAMEID', 'BINARY_WIN', 'W-L-T', 'Power Index FPI',
       'Power Index RK', 'Power Index TREND', 'Power Index OFF',
       'Power Index DEF', 'Power Index ST', 'Ranks SOS', 'Ranks REM SOS',
       'Ranks AVGWP', 'DIVISION_538_Winner', 'ELO WITH QB_538_Winner',
       'ELO CHANGE_538_Winner', 'CURRENT QB ADJ_538_Winner',
       'Points_Diff_538_Winner', 'Playoffs_Prob_538_Winner',
       'Division_Prob_538_Winner', 'Bye_Prob_538_Winner', 'SB_Prob_538_Winner',
       'REAL_WINS2_538_Winner', 'REAL_LOSSES2_538_Winner',
       'REAL_TIES_538_Winner', 'PROJ_WINS4_538_Winner',
       'PROJ_LOSS4_538_Winner', 'EPA/play_RDBMS_Winner'

In [205]:
allTheStats3.groupby(["FINAL_TEAM_NAME"]).count()

,Unnamed: 0,Year,Week_x,Abbr_RDBMS_Winner,Week2,Day,Date,Time,Home_Team,Opponent,...,ESTIM. WINS_FO_ANNUAL_Winner,SCHEDULE RANK_FO_ANNUAL_Winner,VAR._FO_ANNUAL_Winner,TOTAL_GAME_POINTS,TOTAL_GAME_YARDS,TOTAL_GAME_TURNOVERS,PLAYOFFS_BINARY,GAME_TYPE,FINAL_TEAM_NAME_Opp,Power Index DEF_Opp
FINAL_TEAM_NAME,,,,,,,,,,,,,,,,,,,,,
Arizona Cardinals,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
Atlanta Falcons,94,94,94,94,94,94,94,94,94,94,...,94,94,94,94,94,94,94,94,94,94
Baltimore Ravens,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
Buffalo Bills,90,90,90,90,90,90,90,90,90,90,...,90,90,90,90,90,90,90,90,90,90
Carolina Panthers,92,92,92,92,92,92,92,92,92,92,...,92,92,92,92,92,92,92,92,92,92
Chicago Bears,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
Cincinnati Bengals,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
Cleveland Browns,88,88,88,88,88,88,88,88,88,88,...,88,88,88,88,88,88,88,88,88,88
Dallas Cowboys,92,92,92,92,92,92,92,92,92,92,...,92,92,92,92,92,92,92,92,92,92
